In [1]:
import os

import numpy as np
from astropy.coordinates import Angle

from pyuvdata import UVData
from pyuvdata.data import DATA_PATH

In [2]:
testfile = os.path.join(DATA_PATH, 'zen.2458661.23480.HH.uvh5')
uv_raw = UVData()
uv_raw.read_uvh5(testfile)

initial_uvws = uv_raw.uvw_array

uv_raw.set_uvws_from_antenna_positions()

uvw_diff = initial_uvws - uv_raw.uvw_array
print(np.max(np.abs(uvw_diff)))

73.13580521406377


In [3]:
uv_raw.reorder_blts(order="time", minor_order="baseline")
times = np.unique(uv_raw.time_array)
time_set_1, time_set_2 = times[::2], times[1::2]

uv_raw_1 = uv_raw.select(times=time_set_1, inplace=False)
uv_raw_2 = uv_raw.select(times=time_set_2, inplace=False)

uv_raw_1copy = uv_raw_1.copy()


In [4]:
phase_center_radec = (Angle('0d').rad, Angle('-30d').rad)

# phase each half to different spots
uv_raw_1.phase(ra=0,
               dec=0,
               epoch="J2000"
               )
uv_raw_2.phase(ra=phase_center_radec[0],
               dec=phase_center_radec[1],
               epoch="J2000"
               )

# phase original to phase_center_radec
uv_raw.phase(ra=phase_center_radec[0],
             dec=phase_center_radec[1],
             epoch="J2000"
             )


In [5]:
uv_raw_1copy2 = uv_raw_1.copy()
uv_raw_1copy2.unphase_to_drift()

# check that unphasing gets us back to what we started with
vis_diff = uv_raw_1copy.data_array - uv_raw_1copy2.data_array
print(np.max(np.abs(vis_diff)))

assert uv_raw_1copy == uv_raw_1copy2

# phase each the same way
uv_raw_1copy2.phase(ra=phase_center_radec[0],
               dec=phase_center_radec[1],
               epoch="J2000"
               )

uv_raw_1copy.phase(ra=phase_center_radec[0],
               dec=phase_center_radec[1],
               epoch="J2000"
               )

vis_diff = uv_raw_1copy.data_array - uv_raw_1copy2.data_array
print(np.max(np.abs(vis_diff)))

assert uv_raw_1copy == uv_raw_1copy2
assert np.allclose(uv_raw_1copy.data_array, uv_raw_1copy2.data_array)

3.253907165344266e-11
7.587781535024039


In [6]:
uv_out = uv_raw_1.fast_concat(uv_raw_2, axis='blt', phase_center_radec=phase_center_radec)
uv_out.reorder_blts(order='time', minor_order='baseline')

Phasing this UVData object to phase_center_radec


In [7]:
assert uv_out.phase_type == "phased"
uv_out.history = uv_raw.history
assert (uv_out.phase_center_ra, uv_out.phase_center_dec) == phase_center_radec
assert uv_out == uv_raw
